In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib
import warnings
warnings.filterwarnings("ignore")

# 设置图像标签显示中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
# 关掉pandas的warnings
pd.options.mode.chained_assignment = None

## 导入数据

In [2]:
Bitcoin = pd.read_csv('Data\BCHAIN-MKPRU.csv')
Gold = pd.read_csv('Data\LBMA-GOLD.csv')
Bitcoin.columns = ['Date','Value']
Gold.columns = ['Date','Value']
Bitcoin['Date'] = pd.to_datetime(Bitcoin['Date'])
Gold['Date'] = pd.to_datetime(Gold['Date'])

### 交易时间

In [4]:
# 交易数据组合
t = pd.merge(Bitcoin[['Date','Value']],Gold[['Date','Value']],on='Date',how='left')
t.index = t['Date']
all_date = t['Date'].tolist()
# t = t.drop(columns='Date')

# 获得黄金未交易的时间
position = t.isnull().stack()[lambda x: x].index.tolist()
positions = []
for i in range(len(position)):
    positions.append(position[i][0])

# 获得完整时间的交易数据
Bitcoin_Data = t.iloc[:,[0,1]]
Bitcoin_Data.columns = ['Date', 'Value']
Gold_Data= t.iloc[:,[0,2]]
Gold_Data.columns = ['Date', 'Value']

# 将黄金未交易的时间向下填充
Gold_Data = Gold_Data.fillna(method='bfill')

## 时间序列预测

In [114]:
# 预测结束数据
Bitcoin_Data = pd.read_csv('Bitcoin_Data.csv')
Gold_Data = pd.read_csv('Gold_Data.csv')
all_date = pd.to_datetime(Bitcoin_Data['Date']).tolist()

Bitcoin_Data.index = all_date
Gold_Data.index = all_date

# 收益率数据
#Bitcoin_Data['Volatility']= (np.log(Bitcoin_Data['Value'])-np.log(Bitcoin_Data['Value'].shift(1)))/Bitcoin_Data['Value']
#Gold_Data['Volatility']= (np.log(Gold_Data['Value'])-np.log(Gold_Data['Value'].shift(1)))/Gold_Data['Value']
Bitcoin_Data['Volatility'] = np.log(Bitcoin_Data['Value']) / np.log(
    Bitcoin_Data['Value'].shift(1))
Gold_Data['Volatility'] = np.log(Gold_Data['Value']) / np.log(
    Gold_Data['Value'].shift(1))
Bitcoin_Data.head()

,Date,Value,High,Low,TR,ATR,Unit,upper_bound,lower_bound,Value Shift,Predict,Volatility
2016-09-11,2016/9/11,621.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621.650000,NaN
2016-09-12,2016/9/12,609.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621.65,717.434097,0.996975
2016-09-13,2016/9/13,610.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,609.67,706.331257,1.000319
2016-09-14,2016/9/14,608.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,610.92,707.489610,0.999463
2016-09-15,2016/9/15,610.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.82,705.543693,1.000399


## 动态配比系统

In [123]:
start_date = all_date[0] # 起始日期
end_date = all_date[-1] # 结束日期
Vol_Data = pd.DataFrame()
print(start_date)
print(end_date)

2016-09-11 00:00:00
2021-09-10 00:00:00


In [126]:
tt = 30
interval = pd.Timedelta(tt, unit='day')  # 设置间隔日期
cycle = round((end_date - start_date) / interval) - 1
change_list = []
init_date = start_date
for i in range(cycle):
    init_date += interval
    change_list.append(init_date)
print(len(change_list))
Vol_Data['Date'] = change_list

x1 = pd.Timedelta(tt, unit='day')
x2 = pd.Timedelta(1, unit='day')
x3 = pd.Timedelta(5, unit='day')

Bitcoin_profit = []
Bitcoin_mean_profit = []
Bitcoin_var = []
Gold_profit = []
Gold_mean_profit = []
Gold_var = []
corr_list = []
for i in range(len(change_list)):
    t1 = pd.date_range(change_list[i] - x1, change_list[i] - x2, freq='D')
    t2 = pd.date_range(change_list[i] + x2, change_list[i] + x3, freq='D')

    # 过去30天最大收益
    Bitcoin_max_value = Bitcoin_Data.loc[t1]['Value'].max()
    Gold_max_value = Gold_Data.loc[t1]['Value'].max()
    # 未来五天的最大收益
    Bitcoin_predict_value = Bitcoin_Data.loc[t2]['Predict'].max()
    Gold_predict_value = Gold_Data.loc[t2]['Predict'].max()
    # 预计收益率
    #Bitcoin_profit_value = (Bitcoin_predict_value -
    #                       Bitcoin_max_value) / Bitcoin_max_value
    # 预计价比率
    Bitcoin_profit_value = Bitcoin_predict_value / Bitcoin_max_value
    Bitcoin_profit.append(Bitcoin_profit_value)

    #Gold_profit_value = (Gold_predict_value - Gold_max_value) / Gold_max_value
    Gold_profit_value = Gold_predict_value / Gold_max_value
    Gold_profit.append(Gold_profit_value)

    # 过于30天收益率
    Bitcoin_vol = Bitcoin_Data.loc[t1]['Volatility']
    Gold_vol = Gold_Data.loc[t1]['Volatility']

    # 平均收益率
    Bitcoin_mean = Bitcoin_vol.mean()
    Gold_mean = Gold_vol.mean()
    Bitcoin_mean_profit.append(Bitcoin_mean)
    Gold_mean_profit.append(Gold_mean)

    # 收益率方差
    Bitcoin_var_value = Bitcoin_vol.var()
    Gold_var_value = Gold_vol.var()
    Bitcoin_var.append(Bitcoin_var_value)
    Gold_var.append(Gold_var_value)

    # 收益率协方差
    x = pd.DataFrame()
    x['Bitcoin_vol'] = Bitcoin_vol
    x['Gold_vol'] = Gold_vol
    corr_value = x.corr().iloc[0, 1]
    corr_list.append(corr_value)

Vol_Data['Bitcoin_Profit'] = Bitcoin_profit
Vol_Data['Bitcoin_Mean_Profit'] = Bitcoin_mean_profit
Vol_Data['Bitcoin_Var'] = Bitcoin_var
Vol_Data['Gold_Profit'] = Gold_profit
Vol_Data['Gold_Mean_Profit'] = Gold_mean_profit
Vol_Data['Gold_Var'] = Gold_var
Vol_Data['Corr'] = corr_list
# 保存数据
# Vol_Data.to_csv('vol_data.csv')
Vol_Data.head()

60


,Date,Bitcoin_Profit,Bitcoin_Mean_Profit,Bitcoin_Var,Gold_Profit,Gold_Mean_Profit,Gold_Var,Corr
0,2016-10-11,1.178375,0.999965,0.000001,0.938721,0.999758,8.220428e-07,-0.195307
1,2016-11-10,1.104698,1.000799,0.000010,0.970101,1.000081,5.340500e-07,-0.011197
2,2016-12-10,1.116793,1.000334,0.000004,0.915641,0.999549,1.493999e-06,0.232689
3,2017-01-09,0.875224,1.000835,0.000036,1.020633,1.000061,1.076379e-06,0.013685
4,2017-02-08,1.065916,1.000718,0.000029,1.006931,1.000206,1.683486e-06,0.273683


### 动态规划

In [321]:
A = -1
w1, w2 = 0.5, 0.5
e = w1 * Vol_Data['Bitcoin_Profit'] + w2 * Vol_Data['Gold_Profit']
sigma = (pow(w1, 2) * Vol_Data['Bitcoin_Var'] +
         pow(w2, 2) * Vol_Data['Gold_Var']) + w1 * w2 * Vol_Data['Corr']
# 投资效用函数
Vol_Data['U'] = e - 0.5 * A * sigma

In [322]:
from scipy.optimize import minimize
cons = ({
    'type': 'eq',
    'fun': lambda x: -x[0] - x[1] + 1},)
bounds = ((0, 1), (0, 1))


weight = []
for i in range(Vol_Data['Bitcoin_Profit'].shape[0]):
    fun = lambda x: x[0] * Vol_Data['Bitcoin_Profit'][i] + x[1] * Vol_Data[
        'Gold_Profit'][i] - 0.5 * A * (pow(x[0], 2) * Vol_Data['Bitcoin_Var'][i] + pow(
            x[1], 2) * Vol_Data['Gold_Var'][i]) + w1 * w2 * Vol_Data['Corr'][i]
    res = minimize(fun, x0 = (0.5,0.5), bounds=bounds, method='SLSQP', constraints = cons)
    weight.append(res.x)
weight = pd.DataFrame(weight).round(2)
weight.head()

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0


In [323]:
Vol_Data['W0'] = weight.iloc[:,0]
Vol_Data['W1'] = weight.iloc[:,1]

## 海龟交易系统
### 重要指标

In [139]:
from empyrical import value_at_risk


def get_unit(data, T=7, A=20, D1=20, D2=10, money=1000):
    """
    D1=20,D2=10 短线系统
    D1=55,D2=50 长线系统
    """
    # 起始资金
    money_init = money
    # 最近T个交易日的最高价
    data['High'] = talib.MAX(data['Value'], timeperiod=T).shift(1)
    # 最近T个交易日的最低价
    data['Low'] = talib.MIN(data['Value'], timeperiod=T).shift(1)
    # 通过以上数据近真实波动幅度 True Range
    data['TR'] = talib.TRANGE(data['High'], data['Low'], data['Value'])
    # 平均真实波幅
    data['ATR'] = talib.ATR(data['High'],
                            data['Low'],
                            data['Value'],
                            timeperiod=A)
    # 头寸规模单位
    # 交易单位为1 比特币/特洛伊盎司
    data['Unit'] = (money_init * 0.01) / (data['ATR'] * 1)
    data['upper_bound'] = data['Value'].shift(2).rolling(D1).max()
    data['lower_bound'] = data['Value'].shift(2).rolling(D2).min()
    data['Value Shift'] = data['Value'].shift(1)

    # Value at Risk
    VAR = []
    for i in range(A - 1, data.shape[0]):
        VAR.append(
            value_at_risk(data['Value'][i + 1 - A:i], cutoff=0.05))
    VAR = np.array(VAR)
    data['VAR'] = np.hstack((np.zeros(A-1), VAR))
    return data

In [32]:
# 预测结束数据
Bitcoin_Data =  pd.read_csv('Bitcoin_Data.csv')
Gold_Data =  pd.read_csv('Gold_Data.csv')
all_date = pd.to_datetime(Bitcoin_Data['Date']).tolist()

Bitcoin_Data.index = all_date
Gold_Data.index = all_date

# 波动率数据
Bitcoin_Data['Volatility']= (np.log(Bitcoin_Data['Value'])-np.log(Bitcoin_Data['Value'].shift(1)))/Bitcoin_Data['Value']
Gold_Data['Volatility']= (np.log(Gold_Data['Value'])-np.log(Gold_Data['Value'].shift(1)))/Gold_Data['Value']
Bitcoin_Data.head()

,Date,Value,High,Low,TR,ATR,Unit,upper_bound,lower_bound,Value Shift,Predict,Volatility
2016-09-11,2016/9/11,621.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621.650000,NaN
2016-09-12,2016/9/12,609.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621.65,717.434097,-0.000032
2016-09-13,2016/9/13,610.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,609.67,706.331257,0.000003
2016-09-14,2016/9/14,608.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,610.92,707.489610,-0.000006
2016-09-15,2016/9/15,610.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.82,705.543693,0.000004


In [42]:
def reverse(data, f=0.0002, t=0.3):
    capital_residual = 1000.0  # 初始资本价值（美元）
    unit_position = 0  # 初始持有量
    trash = 0  # 手续费
    fees = f  # 佣金
    positions = []  # 保存交易位置
    for i in range(27, data.shape[0]):
        # 我们从28天开始循环，让ATR正确填充
        # 1. 前一天的收盘价超过高价突破位，且未进场交易;
        #    不考虑做空，进场仅买入;
        #    首次建仓为1个单位(unit)
        # 止盈条件

        if (data["Value Shift"].iloc[i] > data["upper_bound"].iloc[i]
                and unit_position == 0):
            # 使用当前价格
            price = data['Value'].iloc[i]
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * data['Unit'].iloc[i]
            unit_position += data['Unit'].iloc[i]
            # 剩余资金量
            capital_residual = capital_residual - capital_buy * (1 + fees)
            trash += capital_buy * fees
            # 总资产
            capital_all = capital_residual + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            stop_loss = price - 2.0 * data['VAR'].iloc[i]
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * data["ATR"].iloc[i]
            positions += [{
                "Date": data['Date'].iloc[i],
                "Price": price,
                "Unit": unit_position
            }]
            print("总资本", capital_all, "剩余资金: ", capital_residual, "交易位置: ",
                  price, "持有份额", unit_position, "交易时间", data['Date'].iloc[i],
                  "止损价位", stop_loss)
        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束

        if unit_position > 0 and (data["Value Shift"].iloc[i] >
                                  positions[-1]['Price']+0.5*data["ATR"].iloc[i]) and (i != data.shape[0] - 1):
            if unit_position > 4 or capital_residual <= 0:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = data['Value'].iloc[i]
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * data['Unit'].iloc[i]
                # 如果剩余资金量充足
                if (capital_buy * (1 + fees)) <= capital_residual:
                    unit_position += data['Unit'].iloc[i]
                    # 剩余资金量
                    capital_residual -= capital_buy * (1 + fees)
                    trash += capital_buy * fees
                # 若不充足
                elif (capital_buy * (1 + fees)) > capital_residual:
                    # 剩下的钱全部购买
                    unit_position += capital_residual / (price * (1 + fees))
                    trash += unit_position * price * fees
                    capital_residual = 0
                capital_all = capital_residual + unit_position * price
                positions += [{
                    "Date": data['Date'].iloc[i],
                    "Price": price,
                    "Unit": unit_position
                }]
                # stop_loss = price - (2.0 - 0.5) * data["ATR"].iloc[i]
                print("总资本", capital_all, "剩余资金: ", capital_residual, "交易位置: ",
                      price, "持有份额", unit_position, "交易时间",
                      data['Date'].iloc[i])
        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position > 0 and (
                data["Value Shift"].iloc[i] < data["lower_bound"].iloc[i]
                or data["Value Shift"].iloc[i] < stop_loss
                or i == data.shape[0] - 1):
            # 使用当前价格全部卖出
            price = data['Value'].iloc[i]
            capital_sell = unit_position * price * (1 - fees)
            trash += unit_position * price * fees
            capital_residual = capital_sell + capital_residual
            capital_all = capital_residual
            print('-' * 50)
            print('M', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position, "时间", data['Date'].iloc[i])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss, unit_position = 0.0, 0.0
    print(trash)
    return positions, capital_all,trash


In [45]:
Bitcoin_Data = get_unit(Bitcoin_Data.copy(), T=7, A=20, D1=20, D2=10, money=1000)
Gold_Data = get_unit(Gold_Data.copy(), T=7, A=20, D1=55, D2=50, money=1000)

In [57]:
# 启动回测 模拟
print("-" * 50)
print("初始资本价值: 1000美元")
print("-" * 50)
Bitcoin_Data = get_unit(Bitcoin_Data.copy(), T=7, A=20, D1=20, D2=30, money=1000)
Bitcoin_Positions, Bitcoin_Capital,trash = reverse(Bitcoin_Data, f=0.0002)

--------------------------------------------------
初始资本价值: 1000美元
--------------------------------------------------
总资本 999.8951313544866 剩余资金:  475.5519037871047 交易位置:  614.74 持有份额 0.8529512111907178 交易时间 2016/10/8 止损价位 591.2919999999999
总资本 1017.8996646934687 剩余资金:  0 交易位置:  635.96 持有份额 1.6005718357970133 交易时间 2016/10/13
--------------------------------------------------
M 当前总资本 1548.5475883104555 卖出价格 967.69 卖出份额 1.6005718357970133 时间 2017/3/19
--------------------------------------------------
总资本 1548.5300444545778 剩余资金:  1460.8107650662587 交易位置:  1181.149838 持有份额 0.07426600467291357 交易时间 2017/4/8 止损价位 911.84761704325
总资本 1552.6260301611142 剩余资金:  1339.2651417539334 交易位置:  1236.63 持有份额 0.17253413584271832 交易时间 2017/4/21
总资本 1561.4654332238701 剩余资金:  1203.6071245678136 交易位置:  1288.02 持有份额 0.27783598752818783 交易时间 2017/4/27
总资本 1573.9340418822458 剩余资金:  1061.308367296481 交易位置:  1333.0 持有份额 0.3845653972886458 交易时间 2017/4/30
总资本 1619.6963847950992 剩余资金:  909.892642587468 交易位置:  1452.

In [59]:
# 启动回测 模拟
print("-" * 50)
print("初始资本价值: 1000美元")
print("-" * 50)
Gold_Data = get_unit(Gold_Data.copy(), T=7, A=20, D1=55, D2=50, money=1000)
Gold_Positions, Gold_Capital,trash = reverse(Gold_Data, f=0.0001)

--------------------------------------------------
初始资本价值: 1000美元
--------------------------------------------------
总资本 999.9472980594987 剩余资金:  472.9278930464991 交易位置:  1214.75 持有份额 0.4338500967384233 交易时间 2017/1/18 止损价位 1168.6511281768633
总资本 1011.7659084533816 剩余资金:  0.9831841644150927 交易位置:  1242.1 持有份额 0.8137692007801035 交易时间 2017/2/8
总资本 1024.0537250765756 剩余资金:  0 交易位置:  1257.2 持有份额 0.8145511653488511 交易时间 2017/2/25
--------------------------------------------------
M 当前总资本 1001.6348496437024 卖出价格 1229.8 卖出份额 0.8145511653488511 时间 2017/5/6
--------------------------------------------------
总资本 1001.5571123781551 剩余资金:  224.18445690494968 交易位置:  1291.0 持有份额 0.602147680459493 交易时间 2017/6/7 止损价位 1257.7855566848016
--------------------------------------------------
M 当前总资本 980.0450613339923 卖出价格 1255.4 卖出份额 0.602147680459493 时间 2017/6/16
--------------------------------------------------
总资本 979.9655751692693 剩余资金:  185.103927939223 交易位置:  1284.4 持有份额 0.6188583363672113 交易时间 2017/8

## 动态投资组合交易

In [363]:
weight_data = Vol_Data[['Date','W0','W1']]
weight_data.index = weight_data['Date']

In [394]:
test_list = np.random.uniform(0,1,3).tolist()+[1]*15+np.random.uniform(0,1,3).tolist()+[1]*20+[0,1]+[1]*17
weight_data['W0'] = np.round(test_list,2)
weight_data['W1'] = 1-weight_data['W0'] 
weight_data.loc['2020-02-23']['W0']=0
weight_data.loc['2020-02-23']['W1']=1
weight_data.loc['2020-03-24']['W0']=1
weight_data.loc['2020-03-24']['W1']=0

In [396]:
Bitcoin_Data = get_unit(Bitcoin_Data.copy(), T=7, A=20, D1=20, D2=300, money=1000)
Gold_Data = get_unit(Gold_Data.copy(), T=7, A=20, D1=55, D2=50, money=1000)

In [397]:
close_date = positions
capital_residual = 1000  # 初始本金
fee_bit, fee_gold = 0.0002, 0.0001  # 佣金
fee_bit_sum, fee_gold_sum = 0, 0  # 总交易成本
sold_change_unit_bit = 0  # 持仓比例改变时，仓位改变份额
sold_change_unit_Gold = 0
W0, W1 = 0.5, 0.5
capital_residual_Bit = capital_residual * W0
capital_residual_Gold = capital_residual * W1
positions_Bit, positions_Gold = [], []
unit_position = [0, 0]
period = pd.date_range(start='2016/09/11', end='2021/09/10')
for i in period:
    if i in weight_data['Date']:

        # 达到时间点，更改权重
        weight_change = weight_data.loc[i]
        W0, W1 = weight_change['W0'], 1 - weight_change['W0']
        # 计算当天持有份额的价值
        capital_hold_Bit = unit_position[0] * Bitcoin_Data.loc[i]['Value']
        capital_hold_Gold = unit_position[1] * Gold_Data.loc[i]['Value']
        # 计算总资本价值
        capital_all = capital_hold_Bit + capital_residual_Bit + capital_hold_Gold + capital_residual_Gold
        # 按比列分配资产组合, 计算需要调整的资本
        capital_change_Bit, capital_change_Gold = capital_all * W0, capital_all * W1
        share_adj_Bit = capital_change_Bit - (capital_hold_Bit +
                                              capital_residual_Bit)
        # 判断是否需要调整资产持有份额
        if share_adj_Bit > 0:
            # 比特币资产比例提高时，黄金资产需要降低份额
            # 判断黄金资产账户，是否剩余足够的资金
            if capital_residual_Gold - share_adj_Bit >= 0:
                # 如果剩余资金足够，划入对应资金比特币账户
                capital_residual_Gold -= share_adj_Bit
                # 比特币账户增持
                capital_residual_Bit += share_adj_Bit
            elif capital_residual_Gold - share_adj_Bit < 0:
                # 黄金资产账户剩余资金不足时，判断如何售出
                if capital_residual_Gold == 0:
                    # 如果黄金账户没有钱，则在下次开市时卖出当前价格对应价值的份额
                    sold_change_unit_Gold = share_adj_Bit / (
                        Gold_Data.loc[i]["Value"] * (1 + fee_gold))
                    # 总仓位同时降低
                    unit_position[1] -= sold_change_unit_Gold
                elif capital_residual_Gold > 0:
                    # 剩下的钱先划走，不够的卖份额
                    sold_change_unit_Gold = (share_adj_Bit -
                                             capital_residual_Gold) / (
                                                 Gold_Data.loc[i]["Value"] *
                                                 (1 + fee_gold))
                    capital_residual_Gold = 0
                    # 总仓位同时降低
                    unit_position[1] -= sold_change_unit_Gold
        elif share_adj_Bit < 0:
            # 黄金资产提高时，比特币资产需要降低份额
            # 判断比特币资产账户，是否剩余足够的资金
            if capital_residual_Bit - np.abs(share_adj_Bit) >= 0:
                # 如果剩余资金足够，划入对应资金黄金账户
                capital_residual_Bit -= np.abs(share_adj_Bit)
                # 黄金账户增持
                capital_residual_Gold += np.abs(share_adj_Bit)
            elif capital_residual_Bit - np.abs(share_adj_Bit) < 0:
                # 比特币资产账户剩余资金不足时，判断如何售出
                if capital_residual_Bit == 0:
                    # 如果比特币账户没有钱，则在下次开市时卖出当前价格对应价值的份额
                    sold_change_unit_bit = np.abs(share_adj_Bit) / (
                        Bitcoin_Data.loc[i]["Value"] * (1 + fee_bit))
                    # 总仓位同时降低
                    unit_position[0] -= sold_change_unit_bit
                elif capital_residual_Bit > 0:
                    # 剩下的钱先划走，不够的卖份额
                    sold_change_unit_bit = (np.abs(share_adj_Bit) -
                                            capital_residual_Bit) / (
                                                Bitcoin_Data.loc[i]["Value"] *
                                                (1 + fee_bit))
                    capital_residual_Bit = 0
                    # 总仓位同时降低
                    unit_position[0] -= sold_change_unit_bit
        print('仓位转换',i,capital_residual_Bit,capital_residual_Gold)
    if i not in close_date:
        # 黄金未闭市时间
        # 黄金交易
        if sold_change_unit_Gold != 0:
            price = Gold_Data.loc[i]['Value']
            capital_residual_Bit += sold_change_unit_Gold * price * (1 -
                                                                     fee_gold)
            fee_gold_sum += sold_change_unit_Gold * price * fee_gold
            print('售出比特币：',sold_change_unit_Gold)
            sold_change_unit_Gold = 0
        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Gold_Data.loc[i]["Value Shift"] > Gold_Data.loc[i]["upper_bound"]
                and unit_position[1] == 0 and capital_residual_Gold > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Gold_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Gold_Data.loc[i]['Unit']
            if capital_buy > capital_residual_Gold:
                unit_position[1] += capital_residual_Gold / (price *
                                                             (1 + fee_bit))
                fee_gold_sum += capital_residual_Gold * fee_gold / (1 +
                                                                    fee_bit)
                capital_buy = capital_residual_Gold
                capital_residual_Gold = 0
            else:
                unit_position[1] += Gold_Data.loc[i]['Unit']
                # 剩余资金量
                capital_residual_Gold = capital_residual_Gold - capital_buy * (
                    1 + fee_gold)
                # 记录消耗费用
                fee_gold_sum += capital_buy * fee_gold
            # 总资产
            capital_all = capital_residual_Gold + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_gold = price - 2.0 * Gold_Data.loc[i]['ATR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Gold_Data.loc[i]["ATR"]
            positions_Gold += [{
                "Date": Gold_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[1]
            }]
            print('黄金建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Gold,
                  "交易位置: ", price, "持有份额", unit_position[1], "交易时间",
                  Gold_Data.loc[i]['Date'], "止损价位", stop_loss_gold)
        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[1] > 0 and capital_residual_Gold > 0 and (
                Gold_Data.loc[i]["Value Shift"] > positions_Gold[-1]['Price'] +
                0.5 * Gold_Data.loc[i]["ATR"]) and (
                    i != pd.to_datetime('2021/09/10')):
            if unit_position[1] > 4 or capital_residual_Gold <= 0:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Gold_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Gold_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_gold)) <= capital_residual_Gold:
                    unit_position[1] += Gold_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Gold -= capital_buy * (1 + fee_gold)
                    # 记录消耗费用
                    fee_gold_sum += capital_buy * fee_gold
                # 若不充足
                elif (capital_buy * (1 + fee_gold)) > capital_residual_Gold:
                    # 剩下的钱全部购买
                    unit_position[1] += capital_residual_Gold / (
                        price * (1 + fee_gold))
                    # 记录消耗费用
                    fee_gold_sum += capital_buy * fee_gold
                    capital_residual_Gold = 0
                capital_all = capital_residual_Gold + unit_position[1] * price
                positions_Gold += [{
                    "Date": Gold_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[1]
                }]
                # stop_loss_gold = price - (2.0 - 0.5) * Gold_Data.loc[i]["ATR"]
            print('黄金加仓', "总资本", capital_all, "剩余资金: ", capital_residual_Gold,
                  "交易位置: ", price, "持有份额", unit_position[1], "交易时间",
                  Gold_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[1] > 0 and (
                Gold_Data.loc[i]["Value Shift"] <
                Gold_Data.loc[i]["lower_bound"]
                or Gold_Data.loc[i]["Value Shift"] < stop_loss_gold
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Gold_Data.loc[i]['Value']
            capital_sell = unit_position[1] * price * (1 - fee_gold)
            fee_gold_sum += unit_position[1] * price * fee_gold
            capital_residual_Gold = capital_sell + capital_residual_Gold
            capital_all = capital_residual_Gold
            print('-' * 50)
            print('M黄金', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[1], "时间", Gold_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_gold, unit_position[1] = 0.0, 0.0

        ###############################################
        ###############################################
        # 比特币交易
        if sold_change_unit_bit != 0:
            price = Bitcoin_Data.loc[i]['Value']
            capital_residual_Gold += sold_change_unit_bit * price * (1 -
                                                                     fee_bit)
            fee_bit_sum += sold_change_unit_bit * price * fee_bit
            print('售出黄金：',sold_change_unit_bit)
            sold_change_unit_bit = 0

        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Bitcoin_Data.loc[i]["Value Shift"] >
                Bitcoin_Data.loc[i]["upper_bound"] and unit_position[0] == 0
                and capital_residual_Bit > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Bitcoin_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Bitcoin_Data.loc[i]['Unit']
            unit_position[0] += Bitcoin_Data.loc[i]['Unit']
            # 剩余资金量
            capital_residual_Bit = capital_residual_Bit - capital_buy * (
                1 + fee_bit)
            # 记录消耗费用
            fee_bit_sum += capital_buy * fee_bit
            # 总资产
            capital_all = capital_residual_Bit + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_bit = price - 2.0 * Bitcoin_Data.loc[i]['ATR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Bitcoin_Data.loc[i]["ATR"]
            positions_Bit += [{
                "Date": Bitcoin_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[0]
            }]
            print('比特币建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'], "止损价位", stop_loss_bit)

        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[0] > 0 and capital_residual_Bit > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] >
                positions_Bit[-1]['Price'] + 0.5 * Bitcoin_Data.loc[i]["ATR"]
        ) and (i != pd.to_datetime('2021/09/10')):
            if unit_position[0] > 4:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Bitcoin_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Bitcoin_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_bit)) <= capital_residual_Bit:
                    unit_position[0] += Bitcoin_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Bit -= capital_buy * (1 + fee_bit)
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                # 若不充足
                elif (capital_buy * (1 + fee_bit)) > capital_residual_Bit:
                    # 剩下的钱全部购买
                    unit_position[0] += capital_residual_Bit / (price *
                                                                (1 + fee_bit))
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                    capital_residual_Bit = 0
                capital_all = capital_residual_Bit + unit_position[0] * price
                positions_Bit += [{
                    "Date": Bitcoin_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[0]
                }]
                # stop_loss = price - (2.0 - 0.5) * Bitcoin_Data.loc[i]["ATR"]
            print("比特币加仓", "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[0] > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] <
                Bitcoin_Data.loc[i]["lower_bound"]
                or Bitcoin_Data.loc[i]["Value Shift"] < stop_loss_bit
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Bitcoin_Data.loc[i]['Value']
            capital_sell = unit_position[0] * price * (1 - fee_bit)
            fee_bit_sum += unit_position[0] * price * fee_bit
            capital_residual_Bit = capital_sell + capital_residual_Bit
            capital_all = capital_residual_Bit
            print('-' * 50)
            print('M比特币', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[0], "时间", Bitcoin_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_bit, unit_position[0] = 0.0, 0.0
    ###############################################
    ###############################################
    else:
        # 黄金闭市时间
        # 仅交易比特币
        if sold_change_unit_bit != 0:
            price = Bitcoin_Data.loc[i]['Value']
            capital_residual_Gold += sold_change_unit_bit * price * (1 -
                                                                     fee_bit)
            fee_bit_sum += sold_change_unit_bit * price * fee_bit
            print('售出黄金：',sold_change_unit_bit)
            sold_change_unit_bit = 0

        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Bitcoin_Data.loc[i]["Value Shift"] >
                Bitcoin_Data.loc[i]["upper_bound"] and unit_position[0] == 0
                and capital_residual_Bit > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Bitcoin_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Bitcoin_Data.loc[i]['Unit']
            unit_position[0] += Bitcoin_Data.loc[i]['Unit']
            # 剩余资金量
            capital_residual_Bit = capital_residual_Bit - capital_buy * (
                1 + fee_bit)
            # 记录消耗费用
            fee_bit_sum += capital_buy * fee_bit
            # 总资产
            capital_all = capital_residual_Bit + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_bit = price - 2.0 * Bitcoin_Data.loc[i]['ATR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Bitcoin_Data.loc[i]["ATR"]
            # positions_Bit[-1]['Price'] + 0.5 * Bitcoin_Data.loc[i]["ATR"]
            positions_Bit += [{
                "Date": Bitcoin_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[0]
            }]
            print('比特币建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'], "止损价位", stop_loss_bit)

        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[0] > 0 and capital_residual_Bit > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] >begin_chance
        ) and (i != pd.to_datetime('2021/09/10')):
            if unit_position[0] > 5 or capital_residual_Bit <= 0:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Bitcoin_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Bitcoin_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_bit)) <= capital_residual_Bit:
                    unit_position[0] += Bitcoin_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Bit -= capital_buy * (1 + fee_bit)
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                # 若不充足
                elif (capital_buy * (1 + fee_bit)) > capital_residual_Bit:
                    # 剩下的钱全部购买
                    unit_position[0] += capital_residual_Bit / (price *
                                                                (1 + fee_bit))
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                    capital_residual_Bit = 0
                capital_all = capital_residual_Bit + unit_position[0] * price
                positions_Bit += [{
                    "Date": Bitcoin_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[0]
                }]
                # stop_loss_bit = price - (2.0 - 0.5) * Bitcoin_Data.loc[i]["ATR"]
            print('比特币加仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[0] > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] <
                Bitcoin_Data.loc[i]["lower_bound"]
                or Bitcoin_Data.loc[i]["Value Shift"] < stop_loss_bit
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Bitcoin_Data.loc[i]['Value']
            capital_sell = unit_position[0] * price * (1 - fee_bit)
            fee_bit_sum += unit_position[0] * price * fee_bit
            capital_residual_Bit += capital_sell
            capital_all = capital_residual_Bit
            print('-' * 50)
            print('M比特币', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[0], "时间", Bitcoin_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_bit, unit_position[0] = 0.0, 0.0

比特币建仓 总资本 499.8951313544866 剩余资金:  -24.44809621289528 交易位置:  614.74 持有份额 0.8529512111907178 交易时间 2016/10/8 止损价位 591.2919999999999
仓位转换 2016-10-11 00:00:00 15.588372637326529 459.9635311497782
比特币加仓 总资本 558.0281078550582 剩余资金:  0 交易位置:  635.96 持有份额 0.877457871336339 交易时间 2016/10/13
仓位转换 2016-11-10 00:00:00 0 459.9635311497782
售出黄金： 0.5440561082502838
仓位转换 2016-12-10 00:00:00 597.3896551566004 255.031850799337
比特币加仓 总资本 853.747751811004 剩余资金:  324.67862475810296 交易位置:  769.08 持有份额 0.6879246984096596 交易时间 2016/12/10
比特币加仓 总资本 859.1397417795274 剩余资金:  42.75403671058933 交易位置:  777.0 持有份额 1.0506894531131765 交易时间 2016/12/11
比特币加仓 总资本 871.4242592834291 剩余资金:  0 交易位置:  788.7 持有份额 1.1048868508728655 交易时间 2016/12/17
仓位转换 2017-01-09 00:00:00 255.031850799337 0.0
比特币加仓 总资本 1260.9996310054996 剩余资金:  151.76797679030113 交易位置:  910.49 持有份额 1.2182798868907934 交易时间 2017/1/9
比特币加仓 总资本 1148.1969302764192 剩余资金:  80.0096243363367 交易位置:  817.91 持有份额 1.305996143756749 交易时间 2017/1/14
比特币加仓 总资本 1150.324483870308

## 代码备份

In [310]:
Bitcoin_Data = get_unit(Bitcoin_Data.copy(), T=7, A=20, D1=20, D2=550, money=1000)
Gold_Data = get_unit(Gold_Data.copy(), T=7, A=20, D1=60, D2=120, money=1000)

In [313]:
close_date = positions
capital_residual = 1000  # 初始本金
fee_bit, fee_gold = 0.0002, 0.0001  # 佣金
fee_bit_sum, fee_gold_sum = 0, 0  # 总交易成本
sold_change_unit_bit = 1  # 持仓比例改变时，仓位改变份额
sold_change_unit_Gold = 1
W0, W1 = 0.8, 0.2
capital_residual_Bit = capital_residual * W0
capital_residual_Gold = capital_residual * W1
positions_Bit, positions_Gold = [], []
unit_position = [0, 0]
period = pd.date_range(start='2016/09/11', end='2021/09/10')
for i in period:
    if i not in close_date:
        # 黄金未闭市时间
        # 黄金交易
        if sold_change_unit_Gold != 0:
            price = Gold_Data.loc[i]['Value']
            capital_residual_Bit += sold_change_unit_Gold * price * (1 -
                                                                     fee_gold)
            fee_gold_sum += sold_change_unit_Gold * price * fee_gold
            sold_change_unit_Gold = 0
        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Gold_Data.loc[i]["Value Shift"] > Gold_Data.loc[i]["upper_bound"]
                and unit_position[1] == 0 and capital_residual_Gold > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Gold_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Gold_Data.loc[i]['Unit']
            if capital_buy > capital_residual_Gold:
                unit_position[1] += capital_residual_Gold / (price *
                                                             (1 + fee_bit))
                fee_gold_sum += capital_residual_Gold * fee_gold / (1 +
                                                                    fee_bit)
                capital_buy = capital_residual_Gold
                capital_residual_Gold = 0
            else:
                unit_position[1] += Gold_Data.loc[i]['Unit']
                # 剩余资金量
                capital_residual_Gold = capital_residual_Gold - capital_buy * (
                    1 + fee_gold)
                # 记录消耗费用
                fee_gold_sum += capital_buy * fee_gold
            # 总资产
            capital_all = capital_residual_Gold + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_gold = price - 2.0 * Gold_Data.loc[i]['VAR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Gold_Data.loc[i]["ATR"]
            positions_Gold += [{
                "Date": Gold_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[1]
            }]
            print('黄金建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Gold,
                  "交易位置: ", price, "持有份额", unit_position[1], "交易时间",
                  Gold_Data.loc[i]['Date'], "止损价位", stop_loss_gold)
        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[1] > 0 and capital_residual_Gold > 0 and (
                Gold_Data.loc[i]["Value Shift"] > positions_Gold[-1]['Price'] +
                0.5 * Gold_Data.loc[i]["ATR"]) and (
                    i != pd.to_datetime('2021/09/10')):
            if unit_position[1] > 4 or capital_residual_Gold <= 0:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Gold_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Gold_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_gold)) <= capital_residual_Gold:
                    unit_position[1] += Gold_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Gold -= capital_buy * (1 + fee_gold)
                    # 记录消耗费用
                    fee_gold_sum += capital_buy * fee_gold
                # 若不充足
                elif (capital_buy * (1 + fee_gold)) > capital_residual_Gold:
                    # 剩下的钱全部购买
                    unit_position[1] += capital_residual_Gold / (
                        price * (1 + fee_gold))
                    # 记录消耗费用
                    fee_gold_sum += capital_buy * fee_gold
                    capital_residual_Gold = 0
                capital_all = capital_residual_Gold + unit_position[1] * price
                positions_Gold += [{
                    "Date": Gold_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[1]
                }]
                # stop_loss_gold = price - (2.0 - 0.5) * Gold_Data.loc[i]["ATR"]
            print('黄金加仓', "总资本", capital_all, "剩余资金: ", capital_residual_Gold,
                  "交易位置: ", price, "持有份额", unit_position[1], "交易时间",
                  Gold_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[1] > 0 and (
                Gold_Data.loc[i]["Value Shift"] <
                Gold_Data.loc[i]["lower_bound"]
                or Gold_Data.loc[i]["Value Shift"] < stop_loss_gold
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Gold_Data.loc[i]['Value']
            capital_sell = unit_position[1] * price * (1 - fee_gold)
            fee_gold_sum += unit_position[1] * price * fee_gold
            capital_residual_Gold = capital_sell + capital_residual_Gold
            capital_all = capital_residual_Gold
            all = unit_position[0]*Bitcoin_Data.loc[i]['Value']+unit_position[1]*Gold_Data.loc[i]['Value']+capital_residual_Bit+capital_residual_Gold
            print('-' * 50)
            print('M黄金', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[1], "时间", Gold_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_gold, unit_position[1] = 0.0, 0.0

        ###############################################
        ###############################################
        # 比特币交易
        if sold_change_unit_bit != 0:
            price = Bitcoin_Data.loc[i]['Value']
            capital_residual_Gold += sold_change_unit_bit * price * (1 -
                                                                     fee_bit)
            fee_bit_sum += sold_change_unit_bit * price * fee_bit
            sold_change_unit_bit = 0

        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Bitcoin_Data.loc[i]["Value Shift"] >
                Bitcoin_Data.loc[i]["upper_bound"] and unit_position[0] == 0
                and capital_residual_Bit > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Bitcoin_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Bitcoin_Data.loc[i]['Unit']
            unit_position[0] += Bitcoin_Data.loc[i]['Unit']
            # 剩余资金量
            capital_residual_Bit = capital_residual_Bit - capital_buy * (
                1 + fee_bit)
            # 记录消耗费用
            fee_bit_sum += capital_buy * fee_bit
            # 总资产
            capital_all = capital_residual_Bit + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_bit = price - 2.0 * Bitcoin_Data.loc[i]['ATR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Bitcoin_Data.loc[i]["ATR"]
            positions_Bit += [{
                "Date": Bitcoin_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[0]
            }]
            print('比特币建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'], "止损价位", stop_loss_bit)

        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[0] > 0 and capital_residual_Bit > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] >
                positions_Bit[-1]['Price'] + 0.5 * Bitcoin_Data.loc[i]["ATR"]
        ) and (i != pd.to_datetime('2021/09/10')):
            if unit_position[0] > 4:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Bitcoin_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Bitcoin_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_bit)) <= capital_residual_Bit:
                    unit_position[0] += Bitcoin_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Bit -= capital_buy * (1 + fee_bit)
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                # 若不充足
                elif (capital_buy * (1 + fee_bit)) > capital_residual_Bit:
                    # 剩下的钱全部购买
                    unit_position[0] += capital_residual_Bit / (price *
                                                                (1 + fee_bit))
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                    capital_residual_Bit = 0
                capital_all = capital_residual_Bit + unit_position[0] * price
                positions_Bit += [{
                    "Date": Bitcoin_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[0]
                }]
                # stop_loss = price - (2.0 - 0.5) * Bitcoin_Data.loc[i]["ATR"]
            print("比特币加仓", "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[0] > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] <
                Bitcoin_Data.loc[i]["lower_bound"]
                or Bitcoin_Data.loc[i]["Value Shift"] < stop_loss_bit
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Bitcoin_Data.loc[i]['Value']
            capital_sell = unit_position[0] * price * (1 - fee_bit)
            fee_bit_sum += unit_position[0] * price * fee_bit
            capital_residual_Bit = capital_sell + capital_residual_Bit
            capital_all = capital_residual_Bit
            print('-' * 50)
            print('M比特币', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[0], "时间", Bitcoin_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_bit, unit_position[0] = 0.0, 0.0
    ###############################################
    ###############################################
    else:
        # 黄金闭市时间
        # 仅交易比特币
        if sold_change_unit_bit != 0:
            price = Bitcoin_Data.loc[i]['Value']
            capital_residual_Gold += sold_change_unit_bit * price * (1 -
                                                                     fee_bit)
            fee_bit_sum += sold_change_unit_bit * price * fee_bit
            sold_change_unit_bit = 0

        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
        if (Bitcoin_Data.loc[i]["Value Shift"] >
                Bitcoin_Data.loc[i]["upper_bound"] and unit_position[0] == 0
                and capital_residual_Bit > 0
                and i != pd.to_datetime('2021/09/10')):
            # 使用当前价格
            price = Bitcoin_Data.loc[i]['Value']
            # 消耗资金量=使用当前价格*一个单位*（1+fees)
            capital_buy = price * Bitcoin_Data.loc[i]['Unit']
            unit_position[0] += Bitcoin_Data.loc[i]['Unit']
            # 剩余资金量
            capital_residual_Bit = capital_residual_Bit - capital_buy * (
                1 + fee_bit)
            # 记录消耗费用
            fee_bit_sum += capital_buy * fee_bit
            # 总资产
            capital_all = capital_residual_Bit + capital_buy
            # 不考虑期货空头，第一次加仓时，止损位置在2N处
            # 止损使用VAR替换ATR
            stop_loss_bit = price - 2.0 * Bitcoin_Data.loc[i]['ATR']
            # 加仓位置在0.5N处
            begin_chance = price + 0.5 * Bitcoin_Data.loc[i]["ATR"]
            positions_Bit += [{
                "Date": Bitcoin_Data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position[0]
            }]
            print('比特币建仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'], "止损价位", stop_loss_bit)

        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
        if unit_position[0] > 0 and capital_residual_Bit > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] >
                positions_Bit[-1]['Price'] + 0.5 * Bitcoin_Data.loc[i]["ATR"]
        ) and (i != pd.to_datetime('2021/09/10')):
            if unit_position[0] > 8 or capital_residual_Bit <= 0:
                # 如果此时仓位达到4unit，或没有子弹，不再加仓
                pass
            else:
                # 使用当前价格
                price = Bitcoin_Data.loc[i]['Value']
                # 购买资金量=使用当前价格*一个单位
                capital_buy = price * Bitcoin_Data.loc[i]['Unit']
                # 如果剩余资金量充足
                if (capital_buy * (1 + fee_bit)) <= capital_residual_Bit:
                    unit_position[0] += Bitcoin_Data.loc[i]['Unit']
                    # 剩余资金量
                    capital_residual_Bit -= capital_buy * (1 + fee_bit)
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                # 若不充足
                elif (capital_buy * (1 + fee_bit)) > capital_residual_Bit:
                    # 剩下的钱全部购买
                    unit_position[0] += capital_residual_Bit / (price *
                                                                (1 + fee_bit))
                    # 记录消耗费用
                    fee_bit_sum += capital_buy * fee_bit
                    capital_residual_Bit = 0
                capital_all = capital_residual_Bit + unit_position[0] * price
                positions_Bit += [{
                    "Date": Bitcoin_Data.loc[i]['Date'],
                    "Price": price,
                    "Unit": unit_position[0]
                }]
                # stop_loss_bit = price - (2.0 - 0.5) * Bitcoin_Data.loc[i]["ATR"]
            print('比特币加仓', "总资本", capital_all, "剩余资金: ", capital_residual_Bit,
                  "交易位置: ", price, "持有份额", unit_position[0], "交易时间",
                  Bitcoin_Data.loc[i]['Date'])

        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
        if unit_position[0] > 0 and (
                Bitcoin_Data.loc[i]["Value Shift"] <
                Bitcoin_Data.loc[i]["lower_bound"]
                or Bitcoin_Data.loc[i]["Value Shift"] < stop_loss_bit
                or i == pd.to_datetime('2021/09/10')):
            # 使用当前价格全部卖出
            price = Bitcoin_Data.loc[i]['Value']
            capital_sell = unit_position[0] * price * (1 - fee_bit)
            fee_bit_sum += unit_position[0] * price * fee_bit
            capital_residual_Bit += capital_sell
            capital_all = capital_residual_Bit
            print('-' * 50)
            print('M比特币', "当前总资本", capital_all, "卖出价格", price, '卖出份额',
                  unit_position[0], "时间", Bitcoin_Data.loc[i]['Date'])
            # 止损位和持仓归零
            print('-' * 50)
            stop_loss_bit, unit_position[0] = 0.0, 0.0
    all = unit_position[0]*Bitcoin_Data.loc[i]['Value']+unit_position[1]*Gold_Data.loc[i]['Value']+capital_residual_Bit+capital_residual_Gold

比特币建仓 总资本 2124.3626713544863 剩余资金:  1600.0194437871044 交易位置:  614.74 持有份额 0.8529512111907178 交易时间 2016/10/8 止损价位 591.2919999999999
比特币加仓 总资本 2142.3564469522394 剩余资金:  1070.6680761134821 交易位置:  635.96 持有份额 1.6851505925510366 交易时间 2016/10/13
比特币加仓 总资本 2171.4048206525827 剩余资金:  631.1799717475499 交易位置:  653.25 持有份额 2.357787751863808 交易时间 2016/10/23
比特币加仓 总资本 2252.5107231043853 剩余资金:  267.4580175636144 交易位置:  687.68 持有份额 2.8865936271823687 交易时间 2016/10/28
比特币加仓 总资本 2279.389160738666 剩余资金:  0 交易位置:  697.01 持有份额 3.270238821162775 交易时间 2016/11/1
黄金建仓 总资本 821.4729680594987 剩余资金:  294.4535630464991 交易位置:  1214.75 持有份额 0.4338500967384233 交易时间 2017/1/18 止损价位 -1086.23
黄金加仓 总资本 833.3093257932312 剩余资金:  0 交易位置:  1242.1 持有份额 0.670887469441455 交易时间 2017/2/8
--------------------------------------------------
M黄金 当前总资本 838.9615091155902 卖出价格 1250.65 卖出份额 0.670887469441455 时间 2017/12/8
--------------------------------------------------
黄金建仓 总资本 838.8938751673882 剩余资金:  162.55439314721968 交易位置:  1314.5 持有份

In [ ]:
def reverse(i,
            data,
            unit_position,
            capital_residual,
            fee,
            fee_sum,
            sold_change_unit,
            capital_residual_other,
            positions,
            kind=0):
    if sold_change_unit != 0:
        price = data.loc[i]['Value']
        capital_residual_other += sold_change_unit * price * (1 - fee)
        fee_sum += sold_change_unit * price * fee
        sold_change_unit = 0

        # 前一天的收盘价超过高价突破位，且未进场交易;
        # 不考虑做空，进场仅买入;
        # 首次建仓为1个单位(unit)
    if (data.loc[i]["Value Shift"] > data.loc[i]["upper_bound"]
            and unit_position == 0 and capital_residual > 0
            and i != pd.to_datetime('2021/09/10')):
        # 使用当前价格
        price = data.loc[i]['Value']
        # 消耗资金量=使用当前价格*一个单位*（1+fees)
        capital_buy = price * data.loc[i]['Unit']
        unit_position += data.loc[i]['Unit']
        # 剩余资金量
        capital_residual = capital_residual - capital_buy * (1 + fee)
        # 记录消耗费用
        fee_sum += capital_buy * fee
        # 总资产
        capital_all = capital_residual + capital_buy
        # 不考虑期货空头，第一次加仓时，止损位置在2N处
        # 止损使用VAR替换ATR
        stop_loss = price - 2.0 * data.loc[i]['VAR']
        # 加仓位置在0.5N处
        begin_chance = price + 0.5 * data.loc[i]["ATR"]
        positions += [{
            "Date": data.loc[i]['Date'],
            "Price": price,
            "Unit": unit_position
        }]
        if kind == 0:
            print('比特币建仓')
        elif kind == 1:
            print("黄金建仓")
        print("总资本", capital_all, "剩余资金: ", capital_residual, "交易位置: ", price,
              "持有份额", unit_position, "交易时间", data.loc[i]['Date'], "止损价位",
              stop_loss)

        # 进场交易以后，仓位大于0，当价格高于上一次交易+0.5*ATR时，加仓。
        # 且满足回测尚未结束
    if unit_position > 0 and (
            data.loc[i]["Value Shift"] > positions[-1]['Price'] +
            0.5 * data.loc[i]["ATR"]) and (i != pd.to_datetime('2021/09/10')):
        if unit_position > 4 or capital_residual <= 0:
            # 如果此时仓位达到4unit，或没有子弹，不再加仓
            pass
        else:
            # 使用当前价格
            price = data.loc[i]['Value']
            # 购买资金量=使用当前价格*一个单位
            capital_buy = price * data.loc[i]['Unit']
            # 如果剩余资金量充足
            if (capital_buy * (1 + fee)) <= capital_residual:
                unit_position += data.loc[i]['Unit']
                # 剩余资金量
                capital_residual -= capital_buy * (1 + fee)
                # 记录消耗费用
                fee_sum += capital_buy * fee
            # 若不充足
            elif (capital_buy * (1 + fee)) > capital_residual:
                # 剩下的钱全部购买
                unit_position += capital_residual / (price * (1 + fee))
                # 记录消耗费用
                fee_sum += capital_buy * fee
                capital_residual = 0

            capital_all = capital_residual + unit_position * price
            positions += [{
                "Date": data.loc[i]['Date'],
                "Price": price,
                "Unit": unit_position
            }]
            # stop_loss = price - (2.0 - 0.5) * data.loc[i]["ATR"]
        if kind == 0:
            print('比特币加仓')
        elif kind == 1:
            print("黄金加仓")
            print("总资本", capital_all, "剩余资金: ", capital_residual, "交易位置: ",
                  price, "持有份额", unit_position, "交易时间", data.loc[i]['Date'])
        # 进场交易以后, 仓位大于0，当价格低于低价突破位，或低于止损价位时，清仓止损
        # 或回测结束时，清仓;
    if unit_position > 0 and (
            data.loc[i]["Value Shift"] < data.loc[i]["lower_bound"]
            or data.loc[i]["Value Shift"] < stop_loss
            or i != pd.to_datetime('2021/09/10')):
        # 使用当前价格全部卖出
        price = data.loc[i]['Value']
        capital_sell = unit_position * price * (1 - fee)
        fee_sum += unit_position * price * fee
        capital_residual = capital_sell + capital_residual
        capital_all = capital_residual
        print('-' * 50)
        if kind == 0:
            print('比特币卖出')
        elif kind == 1:
            print("黄金卖出")
        print('M', "当前总资本", capital_all, "卖出价格", price, '卖出份额', unit_position,
              "时间", data.loc[i]['Date'])
        # 止损位和持仓归零
        print('-' * 50)
        stop_loss, unit_position = 0.0, 0.0

    return unit_position, capital_residual, fee_sum, sold_change_unit, capital_residual_other, positions


In [226]:
close_date = positions
capital_residual = 1000  # 初始本金
fee_bit, fee_gold = 0.0002, 0.0001  # 佣金
fee_bit_sum, fee_gold_sum = 0, 0  # 总交易成本
sold_change_unit_bit = 1  # 持仓比例改变时，仓位改变份额
sold_change_unit_Gold = 1
W0, W1 = 0.5, 0.5
capital_residual_Bit = capital_residual * W0
capital_residual_Gold = capital_residual * W1
positions_Bit, positions_Gold = [], []
unit_position = [0, 0]
period = pd.date_range(start='2016/09/11', end='2021/09/10')


for i in period:
    if i in weight_data['Date']:
        # 达到时间点，更改权重
        weight_change = weight_data.loc[i]
        W0, W1 = weight_change['W0'], 1 - weight_change['W0']
        # 计算当天持有份额的价值
        capital_hold_Bit = unit_position[0] * Bitcoin_Data.loc[i]['Value']
        capital_hold_Gold = unit_position[1] * Gold_Data.loc[i]['Value']
        # 计算总资本价值
        capital_all = capital_hold_Bit + capital_residual_Bit + capital_hold_Gold + capital_residual_Gold
        # 按比列分配资产组合, 计算需要调整的资本
        capital_change_Bit, capital_change_Gold = capital_all * W0, capital_all * W1
        share_adj_Bit = capital_change_Bit - (capital_hold_Bit +
                                              capital_residual_Bit)
        # 判断是否需要调整资产持有份额
        if share_adj_Bit > 0:
            # 比特币资产比例提高时，黄金资产需要降低份额
            # 判断黄金资产账户，是否剩余足够的资金
            if capital_residual_Gold - share_adj_Bit >= 0:
                # 如果剩余资金足够，划入对应资金比特币账户
                capital_residual_Gold -= share_adj_Bit
                # 比特币账户增持
                capital_residual_Bit += share_adj_Bit
            elif capital_residual_Gold - share_adj_Bit < 0:
                # 黄金资产账户剩余资金不足时，判断如何售出
                if capital_residual_Gold == 0:
                    # 如果黄金账户没有钱，则在下次开市时卖出当前价格对应价值的份额
                    sold_change_unit_Gold = share_adj_Bit / (unit_position[1] *
                                                             (1 + fee_gold))
                    # 总仓位同时降低
                    unit_position[1] -= sold_change_unit_Gold
                elif capital_residual_Gold > 0:
                    # 剩下的钱先划走，不够的卖份额
                    capital_residual_Gold = 0
                    sold_change_unit_Gold = (share_adj_Bit -
                                             capital_residual_Gold) / (
                                                 unit_position[1] *
                                                 (1 + fee_gold))
                    # 总仓位同时降低
                    unit_position[1] -= sold_change_unit_Gold
        elif share_adj_Bit < 0:
            # 黄金资产提高时，比特币资产需要降低份额
            # 判断比特币资产账户，是否剩余足够的资金
            if capital_residual_Bit - np.abs(share_adj_Bit) >= 0:
                # 如果剩余资金足够，划入对应资金黄金账户
                capital_residual_Bit -= np.abs(share_adj_Bit)
                # 黄金账户增持
                capital_residual_Gold += np.abs(share_adj_Bit)
            elif capital_residual_Bit - np.abs(share_adj_Bit) < 0:
                # 比特币资产账户剩余资金不足时，判断如何售出
                if capital_residual_Bit == 0:
                    # 如果黄金账户没有钱，则在下次开市时卖出当前价格对应价值的份额
                    sold_change_unit_bit = np.abs(share_adj_Bit) / (unit_position[0] *
                                                            (1 + fee_bit))
                    # 总仓位同时降低
                    unit_position[0] -= sold_change_unit_bit
                elif capital_residual_Bit > 0:
                    # 剩下的钱先划走，不够的卖份额
                    capital_residual_Bit = 0
                    sold_change_unit_bit = (np.abs(share_adj_Bit) -
                                            capital_residual_Bit) / (
                                                unit_position[0] *
                                                (1 + fee_bit))
                    # 总仓位同时降低
                    unit_position[0] -= sold_change_unit_bit
    if i not in close_date:
        # 黄金未闭市时间
        # 黄金交易
        unit_position[
            1], capital_residual_Gold, fee_gold_sum, sold_change_unit_Gold, capital_residual_Bit, positions_Bit = reverse(
                i,
                data=Gold_Data,
                unit_position=unit_position[1],
                capital_residual=capital_residual_Gold,
                fee=fee_gold,
                fee_sum=fee_gold_sum,
                sold_change_unit=sold_change_unit_Gold,
                capital_residual_other=capital_residual_Bit,
                positions=positions_Gold,
                kind=1)
        # 比特币交易
        unit_position[
            0], capital_residual_Bit, fee_bit_sum, sold_change_unit_bit, capital_residual_Gold, positions_Bit = reverse(
                i,
                Bitcoin_Data,
                unit_position[0],
                capital_residual_Bit,
                fee_bit,
                fee_bit_sum,
                sold_change_unit_bit,
                capital_residual_Gold,
                positions_Bit,
                kind=0)

    elif i in close_date:
        # 黄金闭市时间
        # 仅交易比特币
        unit_position[
            0], capital_residual_Bit, fee_bit_sum, sold_change_unit_bit, capital_residual_Gold, positions_Bit = reverse(
                i,
                Bitcoin_Data,
                unit_position[0],
                capital_residual_Bit,
                fee_bit,
                fee_bit_sum,
                sold_change_unit_bit,
                capital_residual_Gold,
                positions_Bit,
                kind=0)

黄金建仓
总资本 2945.946020248087 剩余资金:  2474.0485011179157 交易位置:  1242.1 持有份额 0.3799191040416802 交易时间 2017/2/8 止损价位 -1136.33
--------------------------------------------------
黄金卖出
M 当前总资本 2945.8988304961736 卖出价格 1242.1 卖出份额 0.3799191040416802 时间 2017/2/8
--------------------------------------------------
黄金建仓
总资本 2945.851893690544 剩余资金:  2476.483837392147 交易位置:  1236.8 持有份额 0.37950198601099344 交易时间 2017/2/9 止损价位 -1141.6299999999999
--------------------------------------------------
黄金卖出
M 当前总资本 2945.8049568849137 卖出价格 1236.8 卖出份额 0.37950198601099344 时间 2017/2/9
--------------------------------------------------
黄金建仓
总资本 2945.74714092133 剩余资金:  2367.5875050818963 交易位置:  1253.65 持有份额 0.4611810599764157 交易时间 2017/2/24 止损价位 -1190.85
--------------------------------------------------
黄金卖出
M 当前总资本 2945.689324957746 卖出价格 1253.65 卖出份额 0.4611810599764157 时间 2017/2/24
--------------------------------------------------
黄金建仓
总资本 2945.626765718974 剩余资金:  2320.034378001348 交易位置:  1245.8 持有份额 0.5021611717